# OpenParticle

The main classes of this 'package' are `Fock` and `ParticleOperator`. Their functionality is as follows.

In [1]:
#Imports
from openparticle import Fock, ParticleOperator
from openparticle.qubit_mappings import *

## `Fock` Class

A Fock state in second quantization is a quantum state given by an occupancy representation of some particular mode(s). In quantum field theory, we work mainly in momentum space, so these modes represent the occupied momentum of some particle. For example, for a particle with potential momentum modes $p \in \{0, 1, 2, 3, \dots, N\}$, the corresponding Fock state vector for a particle in momentum mode $2$ would be $|f_{N - 1}, \dots, f_1, f_0 \rangle = |0,  \dots, 0, 1, 0, 0  \rangle$. This is a 'direct encoding' meaning all unoccupied modes are also displayed. A more efficient way to write this state is by only showing occupied orbitals. This same state would be written as $|2\rangle$. This notation is utilized in this package.

In general, we are interested in hadrons, or bound states of Quantum Chromodynamics (QCD). Because of this, we need a representation of our hadronic states in terms of fermions (quarks), antifermions (antiquarks), and bosons (gluons). The way this Fock state is written is: $|f;\bar{f}; b\rangle = |f\rangle \otimes |\bar{f}\rangle\otimes |b\rangle$, where each particle type has its own representation as shown above. Fermions and antifermions obey fermi statistics such that the occupancy of fermions in a particular mode, $n^{f, \bar{f}}_i \in \{0, 1\}$, while gluons obey bose statistics with occupancies in a given mode, $n^b_i \in \{0, 1, 2, 3, \cdots \}$

To define your Fock state, you need to pass in a vector corresponding to occupied fermion modes, occupied antifermion modes, and occupied bosonic modes designated how many bosons exist in each mode. The four parameters to pass when instantiating the class are `f_occ`, `af_occ`, `b_occ`, and an optional `coeff`. The first three are lists corresponding to occupied fermionic, antifermionic, and bosonic modes respectively (note that `b_occ` is a list of tuples where the first value in each tuple is the mode, followed by the number of bosons in this state). `coeff` is a parameter specifying the coefficient in front of the Fock state and is set initially to one. 

In [2]:
#An example state

f_occ = [2] #One fermion in mode 2
af_occ = [1, 3] #One antifermion in mode 1, one in mode 3
b_occ = [(0, 3)] #Three bosons in mode 0

x = Fock(f_occ = f_occ, af_occ = af_occ, b_occ = b_occ)

print(x)

1.0 * |2; 3,1; (0, 3)⟩


### `Fock` Methods:

- `dagger()`: 

Returns the bra corresponding to the Fock state ket.

In [4]:
print(x.dagger())

1.0 * ⟨2; 1,3; (0, 3)|


- display():

Displays the `Fock` with Latex

In [5]:
x.display()

<IPython.core.display.Latex object>

### Related Classes to Fock

### `FockSum`

You can add Fock states the same way that you add kets. Note that you need to make sure the number of modes is the same for each state. 

In [6]:
state_1 = Fock([1], [0], [(0, 2)])
state_2 = Fock([0], [1], [(0,1), (1, 1)])
state = state_1 + state_2
print(state)

1.0 * |1; 0; (0, 2)⟩ + 1.0 * |0; 1; (0, 1),(1, 1)⟩


### `ConjugateFock`

`ConjugateFock` is instantiated in two ways. The first is identical to how `Fock` is instantiated. The second is via the method `.from_state(state: Fock)`, which essentially turns a ket into a bra. 

In [7]:
from openparticle import ConjugateFock

y = ConjugateFock([2], [1, 3], [(0, 3)])
z = ConjugateFock.from_state(x)
print(y)
print(z)

1.0 * ⟨2; 1,3; (0, 3)|
1.0 * ⟨2; 1,3; (0, 3)|


With the `ConjugateFock` class defined, we can now take inner products via `*` or with `conj_state.inner_product(state)`

In [8]:
print(y * z)
print(y.inner_product(z))

1.0
1.0


## `ParticleOperator` Class

The `ParticleOperator` class allows you to define products of fermionic/bosonic creation/annihilation operators that act on Fock states to increase/decrease occupancy numbers. The products of operators that can be created come from the set $\{b_p, b_p^\dagger, d_p, d_p^\dagger, a_p, a_p^\dagger  \}$. For example, the operator $b_1^\dagger b_2 a_2$ acts on a Fock state to create a fermion with momentum $p = 1$, annihilate a fermion with momentum $p = 2$ and annihilate a boson with momentum $p = 2$.

To instantiate an object of the `ParticleOperator` class, you specify a string of operators that are in the product. Each operator in a product has a type of particle, a mode, and whether it is a creation or annihilation operator(`^` corresponds to creation, the absence of this character corresponds to annihilation). The operators in the product are separated by spaces. For example, to instantiate the operator above:

In [3]:
op = ParticleOperator('b1^ b2 a2')
print(op)

1.0*b^†_1b_2a_2


- `.display()`:

Using a `Jupyter Notebook`, we can also print the operator with Latex via the `.display()` method:

In [9]:
op.display()

<IPython.core.display.Latex object>

There are three child classes of `ParticleOperator`.

### `FermionOperator`, `AntifermionOperator`, and `BosonOperator`

These three classes operator similarly to the `ParticleOperator` class; however, they are just single operators by themselves. Thus, we only need to specify the mode and if it is a creation or annihilation operator. Once these operators are defined, products can be taken which return an instance of the `ParticleOperator` class. 

In [4]:
from openparticle import FermionOperator, AntifermionOperator, BosonOperator

In [5]:
op_1 = FermionOperator('1^')
op_2 = FermionOperator('2')
op_3 = BosonOperator('2')

(op_1 * op_2 * op_3).display()


<IPython.core.display.Latex object>

### `ParticleOperatorSum` Class

In [6]:
op_to_sum1 = ParticleOperator('b1^ b2^ a0')
op_to_sum2 = ParticleOperator('b1 d0 a1')
(op_to_sum1 + op_to_sum2).display()

<IPython.core.display.Latex object>

The last import piece of functionality to mention before diving into the qubit mapping section is the ability to act with operators on fock states to produce new states: $\text{op}|\text{Fock state} \rangle = |\text{New Fock state} \rangle$

In [7]:
op_to_act = ParticleOperator('b1^ d2 a0^')
state_to_act_on = Fock([0, 2], [2], [(0, 3), (1, 2)])
output_state = op_to_act * state_to_act_on

display(Latex('$' + op_to_act.op_string +
             ' * ' + state_to_act_on.__str__() + '= ' +\
             output_state.__str__() + '$'))


<IPython.core.display.Latex object>

## Mapping Creation/Annihilation Operators to Qubit Operators

In order to transform creation and annihilation operators to operators that act on qubits, we need to define a suitable map that respects commutation relations for the particle they represent (i.e. $[a_p, a_q^
\dagger] = \delta_{p,q}$ for bosons and $\{b_p, b_q^\dagger \} = \delta_{p,q}$). The canonical way to accomplish this is via the Jordan Wigner transformation for fermions ([McArdle](https://arxiv.org/pdf/1808.10402.pdf)), and the folliwing encoding to be referred to as the unary transformation for bosons ([Somma](https://arxiv.org/pdf/quant-ph/0512209.pdf)). While these are not the only mappings, they are the only currently available for OpenParticle.

- Jordan Wigner Encoding

The Jordan Wigner encoding mapps Fock states to qubit states via: $|f_N,  \dots, f_1, f_0 \rangle \rightarrow |q_N, \dots, q_1, q_0 \rangle$ (note the reversed ordering! This is common notation), where $q_p = f_q \in \{0, 1\}$. The corresponding map for the creation and annihilation operators that respect fermi statistics and commutation relations are: 

\begin{align}
b_p = \frac{1}{2}(X_p - iY_p) \otimes Z_{p - 1} \otimes \dots \otimes Z_0\\
b_p^\dagger = \frac{1}{2}(X_p + iY_p) \otimes Z_{p - 1} \otimes \dots \otimes Z_0

\end{align}

- unary Encoding

We need an alternative mapping for bosons, not just because of the different commutation relations, but also because bosons obey bose statistics, so the occupancy numbers for a given mode can be $f_q \in \{0, 1, 2, \dots \}$. The qubit requirement to map bosonic Fock states to qubit states is greater than that of fermions in general due to these statistics. In order to encode a bosonic system, we first must not only cut off the total number of modes $N$ (just like we do for fermions), but we also must cut off the total number of bosons that can exist in each mode via $N_p^{\text{max}}$. We implicitly assume that each mode has the same cutoff so that the universal cutoff for all modes is $N^\text{max}$.

For each mode $j \in \{0, N\}$, $N^\text{max}$ qubits are needed to encode the occupation number (in a one-hot encoding scheme). For example, if in mode $j = 2$ with $N^\text{max} = 5$, there are 3 bosons that exist in this mode, the Fock state would be encoded as $|0_50_41_30_20_10_0\rangle_2$, where the 2 designates the mode and there is a 1 in position 3 (from left to right it goes 0, 1, 2, etc.) which shows there are 3 bosons in this state $j = 2$ mode. To get the complete state of all $N$ modes, you would have to tensor product the kets between each mode.  

The mapping from bosonic operator to qubit operator is:

\begin{align}

a_p^\dagger = \sum_{n = 0}^{N^\text{max} - 1} \sqrt{n + 1}\sigma_+^{n + 1, j}\sigma_-^{n,j}\\
a_p = \sum_{n = 0}^{N^\text{max} - 1} \sqrt{n + 1}\sigma_-^{n + 1, j}\sigma_+^{n,j}

\end{align}

### `jordan_wigner()`, `unary()`, and `qubit_op_mapping()` Functions

- `jordan_wigner()`:

The `jordan_wigner()` function takes in as a parameter, a single fermionic creation or annihilation operator, as well as a total number of fermionic modes $N$, and returns the qubit state:

In [32]:
op = FermionOperator('3^')
qubit_op = jordan_wigner(op)
op.display()
print("gets mapped to:")
print(qubit_op)

<IPython.core.display.Latex object>

gets mapped to:
 0.500+0.000j XZZZ +
-0.000-0.500j YZZZ


- `unary()`:

The `unary()` function takes in the same parameters, but also takes in the parameter `max_bose_mode_occ` which is $N^\text{max}$.

In [33]:
bos_op = BosonOperator('1')
qubit_bos_op = unary(bos_op, 2)
bos_op.display()
print("gets mapped to:")
print(qubit_bos_op)

<IPython.core.display.Latex object>

gets mapped to:
 0.250+0.000j IXXIII +
 0.000-0.250j IXYIII +
 0.000+0.250j IYXIII +
 0.250+0.000j IYYIII +
 0.354+0.000j XXIIII +
 0.000-0.354j XYIIII +
 0.000+0.354j YXIIII +
 0.354+0.000j YYIIII


- `qubit_op_mapping()`

When we want to map products of these operators, we pass them into the `qubit_op_mapping()` function as follows

In [3]:
op = ParticleOperator('b1^ d1 a0^')
op.display()
qubit_op_mapping(op, 1)

<IPython.core.display.Latex object>

 0.062+0.000j XZXZXX +
 0.000+0.062j XZXZXY +
 0.000-0.062j XZXZYX +
 0.062+0.000j XZXZYY +
 0.000+0.062j XZYZXX +
-0.062+0.000j XZYZXY +
 0.062+0.000j XZYZYX +
 0.000+0.062j XZYZYY +
 0.000-0.062j YZXZXX +
 0.062+0.000j YZXZXY +
-0.062-0.000j YZXZYX +
 0.000-0.062j YZXZYY +
 0.062+0.000j YZYZXX +
 0.000+0.062j YZYZXY +
 0.000-0.062j YZYZYX +
 0.062+0.000j YZYZYY

- `qubit_state_mapping()`:

Lastly, Fock states can be mapped to qubit states with the Jordan Wigner and unary encodings above with: 

In [2]:
state_to_map = Fock([1], [2], [(1, 3)])
state_to_map.display()
qubit_state_mapping(state_to_map, 3)

<IPython.core.display.Latex object>

 1.000+0.000j |1010010000000>